# Interactive Temporal Analysis

---

## 0. Setup and Loading package

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [3]:
import ipyleaflet
import pandas as pd
import geopandas as gpd
import rioxarray as rxr

import geograph
from geograph import geotimeline
from geograph.visualisation import geoviewer
from geograph.constants import UTM35N
from geograph.demo.binder_constants import DATA_DIR, ROIS, ESA_CCI_LEGEND_LINK

---

## 1. Loading Data

In [4]:
# Parse geotif landcover data
chernobyl_path = lambda year: DATA_DIR / "chernobyl" / "esa_cci" / f"esa_cci_{year}_chernobyl.tif" 

# Parse ROIS
rois = gpd.read_file(ROIS)
cez = rois[rois["name"] == "Chernobyl Exclusion Zone"]

In [5]:
def clip_and_reproject(xrdata, clip_geometry=None, to_crs=UTM35N, x_res=300, y_res=300):
    
    if clip_geometry is not None:
        clipped_data = xrdata.rio.clip(clip_geometry)
    else:
        clipped_data = xrdata
        
    if to_crs is not None:
        reprojected_data = clipped_data.rio.reproject(to_crs, resolution=(x_res, y_res))
    else:
        reprojected_data = clipped_data
    
    return reprojected_data

In [7]:
# Loading ESA CCI land cover raster data
years = list(range(2013,2015))
cez_rasters = {year: clip_and_reproject(rxr.open_rasterio(chernobyl_path(year)), clip_geometry=cez.geometry) 
              for year in years
             }

In [8]:
# Loading ESA CCI land cover legend to translate land cover labels to classes
esa_cci_legend = pd.read_csv(ESA_CCI_LEGEND_LINK, delimiter=";", index_col=0)
print(f"There are {len(esa_cci_legend)} classes.")

class_value_to_label = {class_val: row.LCCOwnLabel for class_val, row in esa_cci_legend.iterrows()}

There are 38 classes.


---

## 2. Creating `GeoGraph`

In [9]:
# Polygonising raster and transforming into graph
cez_graphs = {}
for year, raster in cez_rasters.items():
    print(f"Analysing year {year}")
    # Load geograph from the raster data (construction takes ~10s)
    cez_graphs[year] = geograph.GeoGraph(data=raster.data, 
                                transform=raster.rio.transform(), 
                                crs=UTM35N, 
                                mask = raster.data > 0,
                                connectivity=8)
    # Map the ESA CCI land cover class value (int) to the plain text label
    #  to have the plain text labels available in the interactive viewer.
    cez_graphs[year].df.class_label = cez_graphs[year].df.class_label.apply(lambda x: class_value_to_label[x])

Analysing year 2013


Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 2003/2003 [00:10<00:00, 192.82it/s]
Step 2 of 2: Adding edges: 100%|██████████| 2003/2003 [00:00<00:00, 65181.60it/s]


Graph successfully loaded with 2003 nodes and 5140 edges.
Analysing year 2014


Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 1999/1999 [00:10<00:00, 187.49it/s]
Step 2 of 2: Adding edges: 100%|██████████| 1999/1999 [00:00<00:00, 64231.65it/s]

Graph successfully loaded with 1999 nodes and 5117 edges.


---

## 3. Creating `Timeline` and identifying nodes:

In [10]:
cez_timeline = geotimeline.GeoGraphTimeline(cez_graphs)
# Perform node identification
cez_timeline.timestack()
# Classify node dynamics for the year 2014
cez_timeline.calculate_node_dynamics(2014);

---

## 4. Inspect in interactive viewer

In [11]:
viewer = geoviewer.GeoGraphViewer(small_screen=True)
viewer.add_layer(ipyleaflet.basemaps.Esri.WorldImagery)
viewer.add_graph(cez_timeline[2014], name='Chernobyl data', with_components=False)
viewer.enable_graph_controls()
viewer

GeoGraphViewer(center=[51.389167, 30.099444], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…